# Understanding Session 2: Customization and Rate-Limited Concurrency

The `Session` object can be fully customized, including models, model parameters and rate limits, to accustom various usecases. 

Most usefully you can customize: 
- `llmconfig`: the default model parameters for every API call in the session
- `api_service`: rate limit api_service

if you wish to change the default behavior of a session
- you can either pass in a new llmconfig into the Session
- or update the config in the session directly

In [1]:
import lionagi as li

system = "you are a helpful assistant"

In [2]:
llmconfig_ = {...}

# passing in the llmconfig as a dict 
session1 = li.Session(system, llmconfig=llmconfig_)

# or
session1.set_llmconfig(llmconfig_)

# or
session1.llmconfig.update(llmconfig_)

## 1. Default api_service

In [3]:
# import os
# from dotenv import load_dotenv
# load_dotenv()

# let's say you use more than one API key
# api_key2 = os.getenv("OPENAI_API_KEY2")

In [4]:
from lionagi.services import OpenAIService

# let us check the OpenAI api service
service = OpenAIService(
    # api_key = api_key2,           # you can change the api key here - default to OPENAI_API_KEY
    max_requests_per_minute=10,   
    max_tokens_per_minute=10_000
    )

# and then you can pass in the api_service to the session
session3 = li.Session(system, service=service)

In [5]:
# if you wish the rate limit to be applied across sessions, you need to pass in the same api_service when creating the sessions

session4 = li.Session(system, service=service)
session5 = li.Session(system, service=service)
session6 = li.Session(system, service=service)

# now the rate limit is applied across session 3 to 6

## 2. Concurrency

In [6]:
# we will use numpy to generate random numbers for this part
# %pip install numpy

In [7]:
# let us use a simple conditional calculator session as an example
# in this example, we will have two steps in the instruction, first step would be choosing between sum or diff based on a case number
# and second step would be choosing between times or plus based on the sign of the first step

system = "You are asked to perform as a calculator. Return only a numeric value, i.e. int or float, no text."

instruct1 = {
    "sum the absolute values": "provided with 2 numbers, return the sum of their absolute values. i.e. |x|+|y|",}

instruct2 = {
    "diff the absolute values": "provided with 2 numbers, return the difference of absolute values. i.e. |x|-|y|",}

instruct3 = {
    "if previous response is positive": "times 2. i.e. *2", # case 0
    "else": "plus 2. i.e. +2",                              # case 1
}

In [8]:
# create a case and context
case = 0
context = {"x": 7, "y": 3}
instruct = instruct1 if case == 0 else instruct2

In [9]:
from timeit import default_timer as timer
start = timer()
calculator = li.Session(system, dir='data/logs/calculator/')

step1 = await calculator.initiate(instruct, context=context)
step2 = await calculator.followup(instruct3, temperature=0.5)     # you can also modify parameters for each API call

print(f"step1 result: {step1}")
print(f"step2 result: {step2}")

elapsed_time = timer() - start
print(f"run clock time: {elapsed_time:0.2f} seconds")

step1 result: 10
step2 result: 20
run clock time: 1.23 seconds


In [10]:
# now let us run 10 senerios in parallel
import numpy as np

num_iterations = 10

In [11]:
# generate random numbers
ints1 = np.random.randint(-10, 10, size=num_iterations)
ints2 = np.random.randint(0, 10, size=num_iterations)
cases = np.random.randint(0,2, size=num_iterations)

# define a simple parser function
f = lambda i: {"x": str(ints1[i]), "y": str(ints2[i]), "case": str(cases[i])}

In [12]:
# and create the various contexts, l_call (list call) is a helper function to simplify loop
contexts = li.lcall(range(num_iterations), f)

li.lcall(range(num_iterations), lambda i: print(contexts[i]));

{'x': '4', 'y': '7', 'case': '0'}
{'x': '-5', 'y': '9', 'case': '1'}
{'x': '-10', 'y': '2', 'case': '1'}
{'x': '1', 'y': '9', 'case': '0'}
{'x': '8', 'y': '6', 'case': '1'}
{'x': '-8', 'y': '2', 'case': '0'}
{'x': '3', 'y': '4', 'case': '0'}
{'x': '-5', 'y': '3', 'case': '0'}
{'x': '1', 'y': '7', 'case': '1'}
{'x': '-6', 'y': '6', 'case': '0'}


In [13]:
dir = 'data/logs/calculator'

In [14]:
# create a workflow for concurrent execution

async def calculator_workflow(context):
    
    calculator = li.Session(system, dir=dir)       # construct a session instance
    context = context.copy()
    case = int(context.pop("case"))
    
    instruct = instruct1 if case == 0 else instruct2
    await calculator.initiate(instruct, context=context)    # run the steps
    await calculator.followup(instruct3, temperature=0.5)
    
    calculator.messages_to_csv()        # log all messages to csv
    calculator.log_to_csv()             # log all api calls to csv
    return li.lcall(calculator.conversation.responses, lambda i: i['content'])

In [15]:
start = timer()

In [16]:
# use al_call (async list call) to run the workflow concurrently over all senerios
outs = await li.alcall(contexts, calculator_workflow)

5 logs saved to data/logs/calculator/messages_2024-01-03T02_53_07_057680.csv
2 logs saved to data/logs/calculator/llmlog_2024-01-03T02_53_07_058524.csv
5 logs saved to data/logs/calculator/messages_2024-01-03T02_53_07_071238.csv
2 logs saved to data/logs/calculator/llmlog_2024-01-03T02_53_07_071526.csv
5 logs saved to data/logs/calculator/messages_2024-01-03T02_53_07_076506.csv
2 logs saved to data/logs/calculator/llmlog_2024-01-03T02_53_07_076763.csv
5 logs saved to data/logs/calculator/messages_2024-01-03T02_53_07_207447.csv
2 logs saved to data/logs/calculator/llmlog_2024-01-03T02_53_07_208572.csv
5 logs saved to data/logs/calculator/messages_2024-01-03T02_53_07_221342.csv
2 logs saved to data/logs/calculator/llmlog_2024-01-03T02_53_07_222001.csv
5 logs saved to data/logs/calculator/messages_2024-01-03T02_53_07_381796.csv
2 logs saved to data/logs/calculator/llmlog_2024-01-03T02_53_07_382514.csv
5 logs saved to data/logs/calculator/messages_2024-01-03T02_53_07_623054.csv
2 logs save

In [17]:
elapsed_time = timer() - start
print(f"num_workload: {num_iterations}")
print(f"run clock time: {elapsed_time:0.2f} seconds")

num_workload: 10
run clock time: 2.91 seconds


In [18]:
for idx, out in enumerate(outs[:5]):
    print(f"Inputs: {ints1[idx]}, {ints2[idx]}, case: {cases[idx]}\n")
    print(f"Outputs: {out}")
    print("------\n")

Inputs: 4, 7, case: 0

Outputs: ['11', '22']
------

Inputs: -5, 9, case: 1

Outputs: ['4', '8']
------

Inputs: -10, 2, case: 1

Outputs: ['8', '16']
------

Inputs: 1, 9, case: 0

Outputs: ['10', '20']
------

Inputs: 8, 6, case: 1

Outputs: ['2', '4']
------



## 4. Customized api_service concurrent calls

by default, all the session will be created using the same default api_service to ensure rate limit is applied **globally**

But if you would like to have a different api_service and use across sessions, you need to pass in the **same** api_service object during construction

In [19]:
# now let us change the rate limit to check whether it is working
service = OpenAIService(max_requests_per_minute=10, max_tokens_per_minute=10_000)

async def calculator_workflow(context):
    
    calculator = li.Session(system, dir=dir, service=service)       # construct a session instance
    context = context.copy()
    case = int(context.pop("case"))
    instruct = instruct1 if case == 0 else instruct2

    await calculator.initiate(instruct, context=context)    # run the steps
    await calculator.followup(instruct3)
    
    calculator.messages_to_csv(verbose=False)        # log all messages to csv
    calculator.log_to_csv(verbose=False)             # log all api calls to csv

    return li.lcall(calculator.conversation.responses, lambda i: i['content'])

In [20]:
start = timer()

outs = await li.alcall(contexts, calculator_workflow)  

elapsed_time = timer() - start
print(f"num_workload: {num_iterations}")
print(f"run clock time: {elapsed_time:0.2f} seconds")

num_workload: 10
run clock time: 62.94 seconds


In [21]:
for idx, out in enumerate(outs[:5]):
    print(f"Inputs: {ints1[idx]}, {ints2[idx]}, case: {cases[idx]}\n")
    print(f"Outputs: {out}")
    print("------\n")

Inputs: 4, 7, case: 0

Outputs: ['11', '22']
------

Inputs: -5, 9, case: 1

Outputs: ['4', '8']
------

Inputs: -10, 2, case: 1

Outputs: ['8', '16']
------

Inputs: 1, 9, case: 0

Outputs: ['10', '20']
------

Inputs: 8, 6, case: 1

Outputs: ['2', '4']
------

